Mickey-Mouse Example

In [2]:
import pulp

### Set up Mickey-Mouse Data

In [7]:
# Data setup
# Sets
packages = ['P1', 'P2', 'P3']  # Example package IDs
months = ['Jan', 'Feb', 'Mar']  # Example months
years = ['2024']  # Example years
games = ['G1', 'G2', 'G3']  # Example games


# Parameters
C_month = {'P1': 10, 'P2': 20, 'P3': 15}  # Monthly prices 
C_year = {'P1': 100, 'P2': 180, 'P3': 120}  # Yearly prices 


# Example game coverage
P_g = {'G1': ['P1', 'P2'], 'G2': ['P1', 'P3'], 'G3': ['P2', 'P3']}  # Packages that cover each game
m_g = {'G1': 'Jan', 'G2': 'Feb', 'G3': 'Mar'}  # Month of each game
y_g = {'G1': '2024', 'G2': '2024', 'G3': '2024'}  # Year of each game

# Packages that can cover games in a month/year
P_m = {m: [p for p in packages if any(g in P_g and m_g[g] == m for g in games)] for m in months}
P_y = {y: [p for p in packages if any(g in P_g and y_g[g] == y for g in games)] for y in years}




{'Jan': ['P1', 'P2', 'P3'],
 'Feb': ['P1', 'P2', 'P3'],
 'Mar': ['P1', 'P2', 'P3']}

### Define Model

In [10]:
# Model
model = pulp.LpProblem("Streaming_Package_Optimization", pulp.LpMinimize)

# Decision variables
x_month = {p: pulp.LpVariable(f"x_month_{p}", lowBound=0, cat='Integer') for p in packages}
x_year = {p: pulp.LpVariable(f"x_year_{p}", lowBound=0, cat='Integer') for p in packages}
z_month = { (p, m): pulp.LpVariable(f"z_month_{p}_{m}", cat='Binary') for p in packages for m in months }
z_year = { (p, y): pulp.LpVariable(f"z_year_{p}_{y}", cat='Binary') for p in packages for y in years }

# Objective function: Minimize total cost
model += pulp.lpSum(C_month[p] * x_month[p] + C_year[p] * x_year[p] for p in packages)

# Constraints
# 1. Game coverage
for g in games:
    model += pulp.lpSum(z_month[p, m_g[g]] for p in P_g[g]) + pulp.lpSum(z_year[p, y_g[g]] for p in P_g[g]) >= 1

# 2. Monthly subscription activation
for p in packages:
    for m in months:
        model += z_month[p, m] <= x_month[p]

# 3. Yearly subscription activation
for p in packages:
    for y in years:
        model += z_year[p, y] <= x_year[p]

# 4. Covering games by month
for m in months:
    model += pulp.lpSum(z_month[p, m] for p in P_m[m]) >= 1

# 5. Covering games by year
for y in years:
    model += pulp.lpSum(z_year[p, y] for p in P_y[y]) >= 1

In [11]:
# Solve the model
status = model.solve(pulp.PULP_CBC_CMD())

# Output the results
print("Status:", pulp.LpStatus[status])
print("Total Cost:", pulp.value(model.objective))

for p in packages:
    print(f"Package {p}: Monthly subscriptions = {x_month[p].varValue}, Yearly subscriptions = {x_year[p].varValue}")

for p in packages:
    for m in months:
        if z_month[p, m].varValue > 0:
            print(f"Package {p} active in month {m}: {z_month[p, m].varValue}")

for p in packages:
    for y in years:
        if z_year[p, y].varValue > 0:
            print(f"Package {p} active in year {y}: {z_year[p, y].varValue}")

Status: Optimal
Total Cost: 115.0
Package P1: Monthly subscriptions = 0.0, Yearly subscriptions = 1.0
Package P2: Monthly subscriptions = 0.0, Yearly subscriptions = 0.0
Package P3: Monthly subscriptions = 1.0, Yearly subscriptions = 0.0
Package P3 active in month Jan: 1.0
Package P3 active in month Feb: 1.0
Package P3 active in month Mar: 1.0
Package P1 active in year 2024: 1.0
